In [2]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [6]:
# POIs with manually validated coordinates
poi_data = [
    {'name': 'Tacloban_City',        'coord': [125.0015, 11.2434], 'group': 'Urban Core'},
    {'name': 'Ormoc_City',           'coord': [124.6068, 11.0088], 'group': 'Urban Core'},
    {'name': 'Baybay_City',          'coord': [124.7989, 10.6766], 'group': 'Urban Core'},
    {'name': 'Palo',                 'coord': [124.9904, 11.1577], 'group': 'Residential'},
    {'name': 'Alang_Alang',          'coord': [124.8465, 11.2071], 'group': 'Residential'},
    {'name': 'Guiuan',               'coord': [125.7232, 11.0312], 'group': 'Residential'},
    {'name': 'Samar_Forest_Reserve', 'coord': [125.1870, 11.7740], 'group': 'Dark'},
    {'name': 'Mt_Nacolod_Upland',    'coord': [125.3100, 11.4960], 'group': 'Dark'},
    {'name': 'Central_Leyte_Forest', 'coord': [124.8710, 10.8180], 'group': 'Dark'},
    {'name': 'Southern_Leyte_Gulf',  'coord': [125.3222, 10.8153], 'group': 'Dark'},
]

# Color mapping
color_map = {
    'Urban Core': '#FF0000',      # Red
    'Residential': '#FFA500',     # Orange
    'Dark': '#00FF00'             # Green
}

In [7]:
# Load data
df = pd.read_csv("../Datasets/ntl_patch_stats.csv")  # or your path
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['location', 'date'])
df['Valid_pct'] = df['DNB_valid_px'] / df['Total_px'] * 100
df

,DNB_mean,DNB_stdDev,DNB_valid_px,Gap_mean,Gap_stdDev,HQ_hist,QF_hist,Total_px,Valid_pct,date,extent,location
3636,5.565437,NaN,0,2.854094,NaN,"{'0': 0, '2': 0}","{'0': 0, '1': 0}",0,NaN,2013-01-01,1x1,Alang_Alang
3939,4.856050,0.327632,4,1.406627,1.928389,"{'0': 0.3362745098039216, '2': 1.1941176470588...","{'0': 0.3362745098039216, '1': 0.6352941176470...",6,66.666667,2013-01-01,3x3,Alang_Alang
4242,4.626575,1.641227,11,0.640349,1.295799,"{'0': 0.20962566844919783, '2': 1.335472370766...","{'0': 0.20962566844919783, '1': 0.793226381461...",17,64.705882,2013-01-01,5x5,Alang_Alang
3637,NaN,NaN,0,1.882229,NaN,"{'1': 0, '3': 0}",{},0,NaN,2013-01-02,1x1,Alang_Alang
3940,NaN,NaN,0,1.018554,1.052833,"{'1': 0, '3': 0}",{},6,0.000000,2013-01-02,3x3,Alang_Alang
...,...,...,...,...,...,...,...,...,...,...,...,...
604,NaN,NaN,0,27.393932,9.127436,{'1': 0},{},6,0.000000,2013-10-29,3x3,Tacloban_Center
907,NaN,NaN,0,22.647449,16.697030,"{'1': 0, '11': 0, '3': 0}",{},16,0.000000,2013-10-29,5x5,Tacloban_Center
302,NaN,NaN,0,28.277666,NaN,{'2': 0},{},0,NaN,2013-10-30,1x1,Tacloban_Center
605,NaN,NaN,0,27.393932,9.127436,{'2': 0},{},6,0.000000,2013-10-30,3x3,Tacloban_Center


In [8]:
label_map = {'1x1': '1-pixel diameter', '3x3': '3-pixel diameter', '5x5': '5-pixel diameter'}
df['Label'] = df['extent'].map(label_map)
locations = [poi['name'] for poi in poi_data]
extents = list(label_map.keys())

In [9]:
# Prep
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['date', 'location'])

# Define POIs and groups
poi_data = [
    {'name': 'Tacloban_Center',       'group': 'Urban Core'},
    {'name': 'Ormoc_City',            'group': 'Urban Core'},
    {'name': 'Baybay_City',           'group': 'Urban Core'},
    {'name': 'Palo',                  'group': 'Residential'},
    {'name': 'Alang_Alang',           'group': 'Residential'},
    {'name': 'Guiuan',                'group': 'Residential'},
    {'name': 'Samar_Forest_Reserve',  'group': 'Dark'},
    {'name': 'Mt_Nacolod_Upland',     'group': 'Dark'},
    {'name': 'Central_Leyte_Forest',  'group': 'Dark'},
    {'name': 'Southern_Leyte_Gulf',   'group': 'Dark'},
]
group_map = {poi['name']: poi['group'] for poi in poi_data}
df['group'] = df['location'].map(group_map)

# Label map
extent_label_map = {
    '1x1': '1 px radius',
    '3x3': '3 px radius',
    '5x5': '5 px radius'
}

# Colors
group_colors = {
    'Urban Core': 'red',
    'Residential': 'orange',
    'Dark': 'green'
}
group_fill = {
    'Urban Core': 'rgba(255,0,0,0.1)',
    'Residential': 'rgba(255,165,0,0.1)',
    'Dark': 'rgba(0,128,0,0.1)'
}

# Init
fig = make_subplots(rows=1, cols=1)
visibility_map = {'1x1': [], '3x3': [], '5x5': []}

# Build traces per extent
for extent in ['1x1', '3x3', '5x5']:
    df_e = df[df['extent'] == extent]
    if df_e.empty:
        continue

    # Compute group mean ± std
    group_stats = df_e.groupby(['date', 'group'])['Gap_mean'].agg(['mean', 'std']).reset_index()
    group_stats.rename(columns={'mean': 'group_mean', 'std': 'group_std'}, inplace=True)

    for group in df_e['group'].unique():
        gdf = group_stats[group_stats['group'] == group]
        color = group_colors[group]

        # Overall mean ± std for legend
        overall_mean = df_e[df_e['group'] == group]['Gap_mean'].mean()
        overall_std = df_e[df_e['group'] == group]['Gap_mean'].std()
        extent_label = extent_label_map[extent]
        legend_name = f"{group} ({overall_mean:.2f} ± {overall_std:.2f})"

        # STD shaded fill
        fig.add_trace(go.Scatter(
            x=pd.concat([gdf['date'], gdf['date'][::-1]]),
            y=pd.concat([gdf['group_mean'] + gdf['group_std'],
                         (gdf['group_mean'] - gdf['group_std'])[::-1]]),
            fill='toself',
            fillcolor=group_fill[group],
            line=dict(color='rgba(255,255,255,0)'),
            name=f"{group} ({overall_mean:.2f} ± {overall_std:.2f}) [{extent_label}]",
            legendgroup=f'{extent}_{group}',
            showlegend=True,
            visible=(extent == '3x3')
        ))
        visibility_map[extent].append(len(fig.data) - 1)

        # Mean line
        fig.add_trace(go.Scatter(
            x=gdf['date'],
            y=gdf['group_mean'],
            mode='lines',
            name = f"{group} (avg)",
            line=dict(color=color, width=2),
            legendgroup=f'{extent}_{group}',
            showlegend=True,
            visible=(extent == '3x3')
        ))
        visibility_map[extent].append(len(fig.data) - 1)

    # Add faint POIs
    for poi in poi_data:
        loc = poi['name']
        group = poi['group']
        color = group_colors[group]
        df_loc = df_e[df_e['location'] == loc]
        if df_loc.empty:
            continue
        fig.add_trace(go.Scatter(
            x=df_loc['date'],
            y=df_loc['Gap_mean'],
            mode='lines',
            name=f'{loc}',
            line=dict(color=color, width=1, dash='dot'),
            opacity=0.25,
            legendgroup=f'{extent}_{group}',
            showlegend=True,
            visible=(extent == '3x3')
        ))
        visibility_map[extent].append(len(fig.data) - 1)

# Buttons for extent toggle (updates visibility + title)
extent_buttons = []
for extent in ['1x1', '3x3', '5x5']:
    vis = [False] * len(fig.data)
    for i in visibility_map[extent]:
        vis[i] = True
    extent_buttons.append(dict(
        label=extent_label_map[extent],
        method="update",
        args=[
            {"visible": vis},
            {
                "title": f"Gap-Filled NTL Time Series – {extent_label_map[extent]}",
                "yaxis.range": [0, 50],
                "yaxis.title": "Radiance (nW·cm⁻²·sr⁻¹)",
                "xaxis.title": "Date"
            }
        ]
    ))

# Y-axis scale buttons
scale_buttons = [
    dict(label="Linear", method="relayout", args=[{"yaxis.type": "linear"}]),
    dict(label="Log", method="relayout", args=[{"yaxis.type": "log"}])
]

# Final layout
fig.update_layout(
    title=f"Gap-Filled NTL Time Series – {extent_label_map['3x3']}",
    xaxis_title="Date",
    yaxis_title="Radiance (nW·cm⁻²·sr⁻¹)",
    yaxis=dict(range=[0, 50]),
    height=650,
    width=1200,
    paper_bgcolor='rgba(0,0,0,0)', # transparent background 
    # plot_bgcolor='rgba(0,0,0,0)', # transparent plot area,    
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            buttons=extent_buttons,
            showactive=True,
            x=1.05, y=1.12,
            xanchor="left", yanchor="top"
        ),
        dict(
            type="buttons",
            direction="right",
            buttons=scale_buttons,
            showactive=True,
            x=1.05, y=0.12,
            xanchor="left", yanchor="top"
        )
    ]
)
# fig.write_image("../Images/POI_NTL_per_pixel_radius.png", scale=2)
fig.show()

In [24]:
rows = []
extent_map = {'1x1': '1 px radius', '3x3': '3 px radius', '5x5': '5 px radius'}
group_map = {poi['name']: poi['group'] for poi in poi_data}
df['group'] = df['location'].map(group_map)

# Prep for all locations and group averages
location_order = [
    ("Urban Core", "Urban Core (average)"),
    *[(poi['group'], poi['name']) for poi in poi_data if poi['group'] == "Urban Core"],
    ("Residential", "Residential (average)"),
    *[(poi['group'], poi['name']) for poi in poi_data if poi['group'] == "Residential"],
    ("Dark", "Dark (average)"),
    *[(poi['group'], poi['name']) for poi in poi_data if poi['group'] == "Dark"],
]

summary = {loc: {} for _, loc in location_order}

for extent_code, extent_label in extent_map.items():
    df_e = df[df['extent'] == extent_code]

    for group, loc in location_order:
        if "average" in loc:
            df_target = df_e[df_e['group'] == group]
        else:
            df_target = df_e[df_e['location'] == loc]

        if not df_target.empty:
            mean = round(df_target['Gap_mean'].mean(), 2)
            std = round(df_target['Gap_mean'].std(), 2)
            summary[loc][extent_label] = f"{mean} ± {std}"
        else:
            summary[loc][extent_label] = "—"

# Convert to DataFrame
compressed_table = pd.DataFrame([
    {"Location": loc,
     "1 px radius": summary[loc].get('1 px radius', '—'),
     "3 px radius": summary[loc].get('3 px radius', '—'),
     "5 px radius": summary[loc].get('5 px radius', '—')}
    for _, loc in location_order
])

compressed_table

KeyError: 'group'

In [25]:
def generate_extent_summary(df, extent_code):
    extent_label = extent_map[extent_code]
    summary_rows = []

    df_e = df[df['extent'] == extent_code].copy()

    for group, loc in location_order:
        if "average" in loc:
            dfl = df_e[df_e['group'] == group]
        else:
            dfl = df_e[df_e['location'] == loc]

        if dfl.empty:
            continue

        total_days = dfl['date'].nunique()

        # DNB stats
        dnb_mean = round(dfl['DNB_mean'].mean(), 2)
        dnb_std = round(dfl['DNB_stdDev'].mean(), 2)
        dnb_str = f"{dnb_mean} ± {dnb_std}"

        # Gap-Filled stats
        gap_mean = round(dfl['Gap_mean'].mean(), 2)
        gap_std = round(dfl['Gap_stdDev'].mean(), 2)
        gap_str = f"{gap_mean} ± {gap_std}"

        # Avg Valid %
        avg_valid_pct = round(dfl['Valid_pct'].mean(), 1)

        # Days with 0% valid
        zero_days = int((dfl['Valid_pct'] == 0).sum())
        zero_pct = round((zero_days / total_days) * 100, 1)
        zero_display = f"{zero_days} ({zero_pct}%)"

        # Days gap-filled
        gap_days = int((dfl['DNB_mean'].isna() & dfl['Gap_mean'].notna()).sum())
        gap_pct = round((gap_days / total_days) * 100, 1)
        gap_display = f"{gap_days} ({gap_pct}%)"

        # Max dropout
        zero_mask = dfl['Valid_pct'] == 0
        dropout_groups = (zero_mask != zero_mask.shift()).cumsum()
        max_dropout = int(zero_mask.groupby(dropout_groups).sum().max())
        max_dropout_pct = round((max_dropout / total_days) * 100, 1)
        dropout_display = f"{max_dropout} ({max_dropout_pct}%)"

        summary_rows.append({
            "Area": loc,
            "DNB Mean ± SD": dnb_str,
            "Gap-Filled Mean ± SD": gap_str,
            "Avg Valid %": f"{avg_valid_pct}%",
            "Days 0% Valid": zero_display,
            "% Days Gap-Filled": gap_display,
            "Max Dropout (days)": dropout_display
        })

    return pd.DataFrame(summary_rows)

summary_1px = generate_extent_summary(df, '1x1')
summary_1px

NameError: name 'location_order' is not defined

In [27]:
import numpy as np
from ast import literal_eval

def parse_hist(series):
    merged = {}
    for val in series.dropna():
        h = literal_eval(val) if isinstance(val, str) else val
        for k, v in h.items():
            k = int(k)
            merged[k] = merged.get(k, 0) + float(v)
    total = sum(merged.values()) or 1
    return {k: v / total for k, v in merged.items()}

def max_dropout(valid_flags):
    max_run = run = 0
    for v in valid_flags:
        if v == 0:
            run += 1
            max_run = max(max_run, run)
        else:
            run = 0
    return max_run

summary = []
poi_locations = [poi['name'] for poi in poi_data]

for extent_code, extent_label in extent_label_map.items():
    df_e = df[df['extent'] == extent_code].copy()

    for loc in poi_locations:
        dfl = df_e[df_e['location'] == loc].copy()
        if dfl.empty:
            continue

        valid = dfl['DNB_valid_px'].astype(float)
        total = dfl['Total_px'].astype(float)
        valid_flag = (valid > 0).astype(int)

        row = {
            'Location': loc,
            'Extent': extent_label,
            'DNB Mean ± SD': f"{dfl['DNB_mean'].mean():.2f} ± {dfl['DNB_stdDev'].mean():.2f}",
            'Gap-Filled Mean ± SD': f"{dfl['Gap_mean'].mean():.2f} ± {dfl['Gap_stdDev'].mean():.2f}",
            'Valid Days (%)': f"{(valid > 0).sum()} ({valid.mean() / total.mean() * 100:.1f}%)",
            'Gap-Filled Days (%)': f"{(valid == 0).sum()} ({(valid == 0).mean() * 100:.1f}%)",
            'Max Dropout (days)': max_dropout(valid_flag)
        }

        if extent_code == '3x3':
            hq_rel = parse_hist(dfl['HQ_hist'])
            qf_rel = parse_hist(dfl['QF_hist'])

            row.update({
                'Avg. Latest HQ Retrieval (days)': round(sum(k * v for k, v in hq_rel.items()), 2),
                '% HQ = 0 (Same-day)': round(hq_rel.get(0, 0) * 100, 2),
                '% Cloud/Degraded (QF ≥ 2)': round(sum(v for k, v in qf_rel.items() if k >= 2) * 100, 2),
            })

        summary.append(row)

summary_df = pd.DataFrame(summary)
summary_df

,Location,Extent,DNB Mean ± SD,Gap-Filled Mean ± SD,Valid Days (%),Gap-Filled Days (%),Max Dropout (days),Avg. Latest HQ Retrieval (days),% HQ = 0 (Same-day),% Cloud/Degraded (QF ≥ 2)
0,Tacloban_Center,1 px radius,26.15 ± 0.00,31.54 ± 0.00,24 (27.9%),279 (92.1%),110,NaN,NaN,NaN
1,Ormoc_City,1 px radius,18.10 ± 0.00,22.06 ± 0.00,94 (31.0%),209 (69.0%),20,NaN,NaN,NaN
2,Baybay_City,1 px radius,6.47 ± 0.00,9.77 ± 0.00,28 (23.3%),275 (90.8%),111,NaN,NaN,NaN
3,Palo,1 px radius,7.69 ± 0.00,8.65 ± 0.00,36 (30.0%),267 (88.1%),110,NaN,NaN,NaN
4,Alang_Alang,1 px radius,2.12 ± 0.00,2.68 ± 0.00,37 (30.8%),266 (87.8%),110,NaN,NaN,NaN
5,Guiuan,1 px radius,3.91 ± 0.00,5.04 ± 0.00,3 (8.8%),300 (99.0%),150,NaN,NaN,NaN
6,Samar_Forest_Reserve,1 px radius,0.40 ± 0.00,0.16 ± 0.00,102 (37.9%),201 (66.3%),38,NaN,NaN,NaN
7,Mt_Nacolod_Upland,1 px radius,0.50 ± 0.00,0.20 ± 0.00,75 (41.0%),228 (75.2%),79,NaN,NaN,NaN
8,Central_Leyte_Forest,1 px radius,1.01 ± 0.00,0.18 ± 0.00,70 (26.0%),233 (76.9%),42,NaN,NaN,NaN
9,Southern_Leyte_Gulf,1 px radius,0.28 ± 0.00,0.05 ± 0.00,50 (16.5%),253 (83.5%),39,NaN,NaN,NaN


In [29]:
import pandas as pd
import numpy as np
from ast import literal_eval

def parse_hist(series):
    merged = {}
    all_vals = []
    for val in series.dropna():
        h = literal_eval(val) if isinstance(val, str) else val
        all_vals.extend([int(k)] * int(v) for k, v in h.items())
        for k, v in h.items():
            k = int(k)
            merged[k] = merged.get(k, 0) + float(v)
    total = sum(merged.values()) or 1
    return {
        'rel': {k: v / total for k, v in merged.items()},
        'raw': merged,
        'vals': [k for sublist in all_vals for k in sublist]
    }

def max_dropout(valid_flags):
    max_run = run = 0
    for v in valid_flags:
        if v == 0:
            run += 1
            max_run = max(max_run, run)
        else:
            run = 0
    return max_run

summary_dict = {'1x1': [], '3x3': [], '5x5': []}
poi_locations = [poi['name'] for poi in poi_data]

for extent_code in ['1x1', '3x3', '5x5']:
    df_e = df[df['extent'] == extent_code].copy()

    for loc in poi_locations:
        dfl = df_e[df_e['location'] == loc].copy()
        if dfl.empty:
            continue

        row = {'Location': loc}

        if extent_code == '1x1':
            valid_flag = dfl['DNB_mean'].notna().astype(int)
            valid_days = valid_flag.sum()
            total_days = len(dfl)
            gap_filled_days = ((dfl['DNB_mean'].isna()) & (dfl['Gap_mean'].notna())).sum()

            row.update({
                'Valid Days (%)': f"{valid_days} ({valid_days / total_days * 100:.1f}%)",
                'Gap-Filled Days (%)': f"{gap_filled_days} ({gap_filled_days / total_days * 100:.1f}%)",
                'DNB Mean ± STD': f"{dfl['DNB_mean'].mean():.2f} ± {dfl['DNB_stdDev'].mean():.2f}",
                'Gap-Filled Mean ± STD': f"{dfl['Gap_mean'].mean():.2f} ± {dfl['Gap_stdDev'].mean():.2f}",
                'Max Dropout (days)': max_dropout(valid_flag),
                'Median Latest HQ Retrieval (days)': '—',
                '% HQ = 0 (Same-day)': '—'
            })

        else:
            valid = dfl['DNB_valid_px'].astype(float)
            total = dfl['Total_px'].astype(float)
            valid_flag = (valid > 0).astype(int)

            hq = parse_hist(dfl['HQ_hist'])
            qf = parse_hist(dfl['QF_hist'])

            row.update({
                'Valid Days (%)': f"{(valid > 0).sum()} ({valid.mean() / total.mean() * 100:.1f}%)",
                'Gap-Filled Days (%)': f"{(valid == 0).sum()} ({(valid == 0).mean() * 100:.1f}%)",
                'DNB Mean ± STD': f"{dfl['DNB_mean'].mean():.2f} ± {dfl['DNB_stdDev'].mean():.2f}",
                'Gap-Filled Mean ± STD': f"{dfl['Gap_mean'].mean():.2f} ± {dfl['Gap_stdDev'].mean():.2f}",
                'Max Dropout (days)': max_dropout(valid_flag),
                'Median Latest HQ Retrieval (days)': int(np.median(hq['vals'])) if hq['vals'] else '—',
                '% HQ = 0 (Same-day)': round(hq['rel'].get(0, 0) * 100, 2)
            })

        summary_dict[extent_code].append(row)

# Convert to DataFrames
summary_1x1 = pd.DataFrame(summary_dict['1x1'])[
    ['Location', 'Valid Days (%)', 'Gap-Filled Days (%)', 'DNB Mean ± STD',
     'Gap-Filled Mean ± STD', 'Max Dropout (days)', 'Median Latest HQ Retrieval (days)', '% HQ = 0 (Same-day)']
]
summary_3x3 = pd.DataFrame(summary_dict['3x3'])[summary_1x1.columns]
summary_5x5 = pd.DataFrame(summary_dict['5x5'])[summary_1x1.columns]

In [30]:
summary_1x1

,Location,Valid Days (%),Gap-Filled Days (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_Center,100 (33.0%),203 (67.0%),26.15 ± 0.00,31.54 ± 0.00,21,—,—
1,Ormoc_City,95 (31.4%),208 (68.6%),18.10 ± 0.00,22.06 ± 0.00,20,—,—
2,Baybay_City,95 (31.4%),208 (68.6%),6.47 ± 0.00,9.77 ± 0.00,36,—,—
3,Palo,104 (34.3%),199 (65.7%),7.69 ± 0.00,8.65 ± 0.00,38,—,—
4,Alang_Alang,116 (38.3%),187 (61.7%),2.12 ± 0.00,2.68 ± 0.00,20,—,—
5,Guiuan,98 (32.3%),205 (67.7%),3.91 ± 0.00,5.04 ± 0.00,26,—,—
6,Samar_Forest_Reserve,114 (37.6%),189 (62.4%),0.40 ± 0.00,0.16 ± 0.00,23,—,—
7,Mt_Nacolod_Upland,117 (38.6%),186 (61.4%),0.50 ± 0.00,0.20 ± 0.00,21,—,—
8,Central_Leyte_Forest,83 (27.4%),220 (72.6%),1.01 ± 0.00,0.18 ± 0.00,25,—,—
9,Southern_Leyte_Gulf,54 (17.8%),249 (82.2%),0.28 ± 0.00,0.05 ± 0.00,39,—,—


In [31]:
summary_3x3

,Location,Valid Days (%),Gap-Filled Days (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_Center,100 (31.1%),203 (67.0%),23.15 ± 9.14,29.13 ± 12.86,21,0,73.41
1,Ormoc_City,97 (29.5%),206 (68.0%),13.91 ± 6.58,16.77 ± 6.62,20,6,70.77
2,Baybay_City,95 (27.3%),208 (68.6%),4.85 ± 2.55,7.77 ± 2.95,36,0,55.44
3,Palo,107 (33.1%),196 (64.7%),5.85 ± 3.47,7.26 ± 3.73,21,0,67.89
4,Alang_Alang,118 (35.2%),185 (61.1%),1.37 ± 0.85,1.17 ± 1.28,20,0,72.45
5,Guiuan,98 (29.3%),205 (67.7%),3.10 ± 1.24,3.93 ± 1.54,26,0,63.25
6,Samar_Forest_Reserve,121 (36.4%),182 (60.1%),0.46 ± 0.25,0.16 ± 0.06,23,1,72.64
7,Mt_Nacolod_Upland,120 (36.0%),183 (60.4%),0.55 ± 0.32,0.20 ± 0.08,21,0,73.85
8,Central_Leyte_Forest,90 (24.9%),213 (70.3%),0.96 ± 0.32,0.18 ± 0.06,22,1,57.03
9,Southern_Leyte_Gulf,59 (15.8%),244 (80.5%),0.23 ± 0.10,0.05 ± 0.02,39,2,53.23


In [32]:
summary_5x5

,Location,Valid Days (%),Gap-Filled Days (%),DNB Mean ± STD,Gap-Filled Mean ± STD,Max Dropout (days),Median Latest HQ Retrieval (days),% HQ = 0 (Same-day)
0,Tacloban_Center,106 (30.3%),197 (65.0%),17.06 ± 11.48,22.75 ± 15.75,21,1,47.18
1,Ormoc_City,103 (27.4%),200 (66.0%),10.22 ± 7.15,11.51 ± 8.33,20,1,44.96
2,Baybay_City,106 (26.5%),197 (65.0%),3.16 ± 2.48,4.42 ± 4.11,36,1,35.58
3,Palo,108 (32.8%),195 (64.4%),4.14 ± 3.48,4.89 ± 4.06,21,1,67.21
4,Alang_Alang,123 (35.3%),180 (59.4%),0.94 ± 0.76,0.69 ± 0.89,20,2,56.53
5,Guiuan,107 (30.0%),196 (64.7%),2.06 ± 1.52,2.35 ± 2.04,26,1,38.57
6,Samar_Forest_Reserve,127 (36.2%),176 (58.1%),0.52 ± 0.31,0.17 ± 0.08,23,1,56.68
7,Mt_Nacolod_Upland,124 (35.0%),179 (59.1%),0.59 ± 0.42,0.20 ± 0.09,21,0,63.49
8,Central_Leyte_Forest,99 (24.9%),204 (67.3%),1.13 ± 0.49,0.18 ± 0.07,22,2,35.88
9,Southern_Leyte_Gulf,65 (15.8%),238 (78.5%),0.23 ± 0.11,0.05 ± 0.03,39,2,36.12


In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# --- Setup ---
extent_codes = ['1x1', '3x3', '5x5']
extent_label_map = {'1x1': '1 px radius', '3x3': '3 px radius', '5x5': '5 px radius'}

poi_data = [
    {'name': 'Tacloban_Center'}, {'name': 'Ormoc_City'}, {'name': 'Baybay_City'},
    {'name': 'Palo'}, {'name': 'Alang_Alang'}, {'name': 'Guiuan'},
    {'name': 'Samar_Forest_Reserve'}, {'name': 'Mt_Nacolod_Upland'},
    {'name': 'Central_Leyte_Forest'}, {'name': 'Southern_Leyte_Gulf'}
]
locations = [p['name'] for p in poi_data]

# --- Create figure ---
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.20, 0.70],
    shared_xaxes=True,
    vertical_spacing=0.10,
    subplot_titles=("Observed Pixel Coverage (%)", "NTL Radiance Time Series")
)

traces = []
trace_meta = []

# --- Loop through extents and locations ---
for extent in extent_codes:
    for loc in locations:
        dfl = df[(df['extent'] == extent) & (df['location'] == loc)]
        if dfl.empty:
            continue
        dnb_patch = dfl[dfl['DNB_stdDev'].notnull() & (dfl['DNB_valid_px'] > 1)]

        # Coverage bar
        traces.append(go.Bar(
            x=dfl['date'],
            y=dfl['Valid_pct'],
            name='Coverage',
            marker_color='green',
            opacity=0.3,
            visible=(extent == '3x3' and loc == locations[0])
        ))
        trace_meta.append((extent, loc, 'coverage'))

        # Gap line
        traces.append(go.Scatter(
            x=dfl['date'], y=dfl['Gap_mean'],
            mode='lines',
            name='Gap-Filled',
            line=dict(color='red'),
            visible=(extent == '3x3' and loc == locations[0]),
            legendgroup=f'gap_{loc}'
        ))
        trace_meta.append((extent, loc, 'gap'))

        # # Gap fill
        # traces.append(go.Scatter(
        #     x=pd.concat([dfl['date'], dfl['date'][::-1]]),
        #     y=pd.concat([
        #         dfl['Gap_mean'] + dfl['Gap_stdDev'],
        #         (dfl['Gap_mean'] - dfl['Gap_stdDev'])[::-1]
        #     ]),
        #     fill='toself',
        #     fillcolor='rgba(255,0,0,0.1)',
        #     line=dict(color='rgba(255,255,255,0)'),
        #     hoverinfo='skip',
        #     showlegend=False,
        #     visible=(extent == '3x3' and loc == locations[0])
        # ))
        # trace_meta.append((extent, loc, 'gap_fill'))

        # DNB line
        traces.append(go.Scatter(
            x=dfl['date'], y=dfl['DNB_mean'],
            mode='lines+markers',
            name='Observed',
            line=dict(color='blue'),
            marker=dict(size=4),
            visible=(extent == '3x3' and loc == locations[0]),
            legendgroup=f'dnb_{loc}'
        ))
        trace_meta.append((extent, loc, 'dnb'))

        # # DNB fill
        # traces.append(go.Scatter(
        #     x=pd.concat([dnb_patch['date'], dnb_patch['date'][::-1]]),
        #     y=pd.concat([
        #         dnb_patch['DNB_mean'] + dnb_patch['DNB_stdDev'],
        #         (dnb_patch['DNB_mean'] - dnb_patch['DNB_stdDev'])[::-1]
        #     ]),
        #     fill='toself',
        #     fillcolor='rgba(0,0,255,0.1)',
        #     line=dict(color='rgba(255,255,255,0)'),
        #     hoverinfo='skip',
        #     showlegend=False,
        #     visible=(extent == '3x3' and loc == locations[0])
        # ))
        # trace_meta.append((extent, loc, 'dnb_fill'))

# --- Add all traces to figure ---
for i, trace in enumerate(traces):
    fig.add_trace(trace, row=1 if trace_meta[i][2] == 'coverage' else 2, col=1)

# --- Dropdown for locations ---
location_buttons = []
for loc in locations:
    vis = [(e == '3x3' and l == loc) for e, l, _ in trace_meta]
    location_buttons.append(dict(
        label=loc,
        method='update',
        args=[{'visible': vis},
              {'title': f'NTL Radiance – 3x3 – {loc}'}]
    ))

# --- Buttons for extents ---
extent_buttons = []
for extent in extent_codes:
    vis = [(e == extent and l == locations[0]) for e, l, _ in trace_meta]
    extent_buttons.append(dict(
        label=extent_label_map[extent],
        method='update',
        args=[{'visible': vis},
              {'title': f'NTL Radiance – {extent_label_map[extent]} – {locations[0]}'}]
    ))

# --- Haiyan Landfall ---
fig.add_vline(
    x=pd.to_datetime("2013-11-08"),
    line_width=2, line_dash="dash", line_color="blue"
)
fig.add_annotation(
    x="2013-11-08", y=1.02, yref='paper',
    text="Haiyan Landfall",
    showarrow=False,
    font=dict(color="blue"),
    bgcolor="white",
    bordercolor="blue",
    borderwidth=1,
    xanchor="left"
)

# --- Layout ---
fig.update_layout(
    updatemenus=[
        dict(type="dropdown", direction="down", buttons=location_buttons,
             showactive=True, x=1.1, xanchor="center", y=0.6, yanchor="bottom"),
        dict(type="buttons", direction="right", buttons=extent_buttons,
             showactive=True, x=1.0, y=1.15, xanchor="right", yanchor="top")
    ],
    height=700,
    width=1200,
    paper_bgcolor='rgba(0,0,0,0)', # transparent background 

    template='plotly_white',
    xaxis2=dict(title='Date', rangeslider=dict(visible=True)),
    yaxis=dict(title='Px Coverage (%)', range=[0, 100]),
    yaxis2=dict(title='Radiance (nW·cm⁻²·sr⁻¹)') #range=[0, 10])
)

fig.show()

NameError: name 'df' is not defined

In [14]:
# # Define POI order
# poi_data = [
#     {'name': 'Tacloban_Center'}, {'name': 'Ormoc_City'}, {'name': 'Baybay_City'},
#     {'name': 'Palo'}, {'name': 'Alang_Alang'}, {'name': 'Guiuan'},
#     {'name': 'Samar_Forest_Reserve'}, {'name': 'Mt_Nacolod_Upland'},
#     {'name': 'Central_Leyte_Forest'}, {'name': 'Southern_Leyte_Gulf'}
# ]
# locations = [poi['name'] for poi in poi_data]

# # Use only 3x3 for now
# df = df[df['extent'] == '3x3']
# max_gap_radiance = (df['Gap_mean'] + df['Gap_stdDev']).max()

# # Create subplot
# fig = make_subplots(
#     rows=2, cols=1,
#     row_heights=[0.20, 0.70],
#     shared_xaxes=True,
#     vertical_spacing=0.10,
#     subplot_titles=("Observed Pixel Coverage (%)", "NTL Radiance Time Series")
# )

# # Add all traces for all locations (3x3)
# traces_per_location = {}
# for loc in locations:
#     dfl = df[df['location'] == loc]
#     if dfl.empty:
#         continue

#     dnb_patch = dfl[dfl['DNB_stdDev'].notnull() & (dfl['DNB_valid_px'] > 1)]

#     coverage_bar = go.Bar(
#         x=dfl['date'],
#         y=dfl['Valid_pct'],
#         name='Coverage – 3x3',
#         marker_color='green',
#         opacity=0.3,
#         visible=(loc == locations[0])
#     )

#     gap_line = go.Scatter(
#         x=dfl['date'],
#         y=dfl['Gap_mean'],
#         mode='lines',
#         name='Gap-Filled – 3x3',
#         line=dict(color='red'),
#         visible=(loc == locations[0]),
#         legendgroup=f'gap_{loc}'
#     )

#     gap_fill = go.Scatter(
#         x=pd.concat([dfl['date'], dfl['date'][::-1]]),
#         y=pd.concat([
#             dfl['Gap_mean'] + dfl['Gap_stdDev'],
#             (dfl['Gap_mean'] - dfl['Gap_stdDev'])[::-1]
#         ]),
#         fill='toself',
#         fillcolor='rgba(255,0,0,0.1)',
#         line=dict(color='rgba(255,255,255,0)'),
#         hoverinfo='skip',
#         showlegend=False,
#         visible=(loc == locations[0]),
#         legendgroup=f'gap_{loc}'
#     )

#     dnb_line = go.Scatter(
#         x=dfl['date'],
#         y=dfl['DNB_mean'],
#         mode='lines+markers',
#         name='Observed – 3x3',
#         line=dict(color='blue'),
#         marker=dict(size=4),
#         visible=(loc == locations[0]),
#         legendgroup=f'dnb_{loc}'
#     )

#     dnb_fill = go.Scatter(
#         x=pd.concat([dnb_patch['date'], dnb_patch['date'][::-1]]),
#         y=pd.concat([
#             dnb_patch['DNB_mean'] + dnb_patch['DNB_stdDev'],
#             (dnb_patch['DNB_mean'] - dnb_patch['DNB_stdDev'])[::-1]
#         ]),
#         fill='toself',
#         fillcolor='rgba(0,0,255,0.1)',
#         line=dict(color='rgba(255,255,255,0)'),
#         hoverinfo='skip',
#         showlegend=False,
#         visible=(loc == locations[0]),
#         legendgroup=f'dnb_{loc}'
#     )

#     # Add to plot
#     fig.add_trace(coverage_bar, row=1, col=1)
#     fig.add_trace(gap_fill, row=2, col=1)
#     fig.add_trace(gap_line, row=2, col=1)
#     fig.add_trace(dnb_fill, row=2, col=1)
#     fig.add_trace(dnb_line, row=2, col=1)

#     traces_per_location[loc] = 5  # store how many traces per location

# # Create dropdown
# buttons = []
# cursor = 0
# for loc in locations:
#     vis = [False] * (len(locations) * 5)
#     for i in range(5):
#         vis[cursor + i] = True
#     cursor += 5

#     buttons.append(dict(
#         label=loc,
#         method='update',
#         args=[
#             {'visible': vis},
#             {'title': f'NTL Radiance – 3x3 – {loc}'}
#         ]
#     ))

# # Add Haiyan vline
# fig.add_vline(
#     x=pd.to_datetime("2013-11-08"),
#     line_width=2,
#     line_dash="dash",
#     line_color="blue"
# )
# fig.add_annotation(
#     x="2013-11-08", y=1.02, yref='paper',
#     text="Haiyan Landfall",
#     showarrow=False,
#     font=dict(color="blue"),
#     bgcolor="white",
#     bordercolor="blue",
#     borderwidth=1,
#     xanchor="left"
# )

# # Final layout
# fig.update_layout(
#     updatemenus=[dict(
#         type="dropdown",
#         direction="down",
#         buttons=buttons,
#         showactive=True,
#         x=1.1, xanchor="center",
#         y=0.6, yanchor="bottom"
#     )],
#     height=650,
#     width=1200,
#     paper_bgcolor='rgba(0,0,0,0)', # transparent background 
#     xaxis2=dict(title='Date', rangeslider=dict(visible=True)),
#     yaxis=dict(title='Px Coverage (%)', range=[0, 100]),
#     yaxis2=dict(title='Radiance (nW·cm⁻²·sr⁻¹)'),
#     template='plotly_white'
# )

# fig.show()

In [21]:
summary_rows = []
extent_label_map = {
    '1x1': '1 px radius',
    '3x3': '3 px radius',
    '5x5': '5 px radius'
}
poi_locations = [poi['name'] for poi in poi_data]

for extent_code, extent_label in extent_label_map.items():
    df_e = df[df['extent'] == extent_code]
    
    for loc in poi_locations:
        dfl = df_e[df_e['location'] == loc]
        if dfl.empty:
            continue
        
        # DNB mean ± std
        dnb_mean = round(dfl['DNB_mean'].mean(), 2)
        dnb_std = round(dfl['DNB_stdDev'].mean(), 2)

        # Gap mean ± std
        gap_mean = round(dfl['Gap_mean'].mean(), 2)
        gap_std = round(dfl['Gap_stdDev'].mean(), 2)

        # Valid percent summary
        valid_pct_avg = round(dfl['Valid_pct'].mean(), 1)
        valid_pct_zero_days = (dfl['Valid_pct'] == 0).sum()

        summary_rows.append({
            "Location": loc,
            "Extent": extent_label,
            "DNB Mean ± SD": f"{dnb_mean} ± {dnb_std}",
            "Gap Mean ± SD": f"{gap_mean} ± {gap_std}",
            "Avg Valid %": f"{valid_pct_avg}%",
            "Days 0% Valid": valid_pct_zero_days
        })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(by=["Location", "Extent"])
summary_df

,Location,Extent,DNB Mean ± SD,Gap Mean ± SD,Avg Valid %,Days 0% Valid
4,Alang_Alang,1 px radius,2.12 ± 0.0,2.68 ± 0.0,30.8%,83
14,Alang_Alang,3 px radius,1.37 ± 0.85,1.17 ± 1.28,35.3%,185
24,Alang_Alang,5 px radius,0.94 ± 0.76,0.69 ± 0.89,35.3%,180
2,Baybay_City,1 px radius,6.47 ± 0.0,9.77 ± 0.0,23.3%,92
12,Baybay_City,3 px radius,4.85 ± 2.55,7.77 ± 2.95,28.4%,208
22,Baybay_City,5 px radius,3.16 ± 2.48,4.42 ± 4.11,26.6%,197
8,Central_Leyte_Forest,1 px radius,1.01 ± 0.0,0.18 ± 0.0,26.0%,199
18,Central_Leyte_Forest,3 px radius,0.96 ± 0.32,0.18 ± 0.06,24.8%,213
28,Central_Leyte_Forest,5 px radius,1.13 ± 0.49,0.18 ± 0.07,24.8%,204
5,Guiuan,1 px radius,3.91 ± 0.0,5.04 ± 0.0,8.8%,31


In [ ]:
summary_df

In [19]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose

# --- Prep ---
group_colors = {
    'Urban Core': 'red',
    'Residential': 'orange',
    'Dark': 'green'
}
group_map = {poi['name']: poi['group'] for poi in poi_data}

df['date'] = pd.to_datetime(df['date'])
df = df[df['extent'] == '3x3'].copy()
df['group'] = df['location'].map(group_map)

rolling_options = [1, 4, 7, 14, 28, 30]

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=False,
    row_heights=[0.65, 0.35],
    vertical_spacing=0.1,
    subplot_titles=["Trend (Group Avg + Individual Faint Lines)", "Seasonality (1-Cycle Snapshot)"]
)

trace_refs = []
slider_steps = []

for i, period in enumerate(rolling_options):
    show = (i == 0)

    for group in df['group'].unique():
        color = group_colors[group]
        df_group = df[df['group'] == group]
        ts_avg = df_group.groupby('date')['Gap_mean'].mean().dropna()

        if len(ts_avg) < period * 2:
            continue

        try:
            decomposition = seasonal_decompose(ts_avg, model='additive', period=period)
        except:
            continue

        trend_avg = decomposition.trend
        seasonal_avg = decomposition.seasonal

        # --- Trend Line (Main) ---
        fig.add_trace(go.Scatter(
            x=trend_avg.index, y=trend_avg,
            name=f"{group} Trend",
            line=dict(color=color, width=2),
            legendgroup=f'{group}_trend',
            visible=show,
            showlegend=True
        ), row=1, col=1)
        trace_refs.append((period, len(fig.data) - 1))

        # --- Individual POIs (Faint) ---
        for loc in df_group['location'].unique():
            df_loc = df_group[df_group['location'] == loc].sort_values('date')
            ts = df_loc.set_index('date')['Gap_mean'].dropna()
            if len(ts) < period * 2:
                continue
            try:
                decomposition = seasonal_decompose(ts, model='additive', period=period)
            except:
                continue

            trend = decomposition.trend

            fig.add_trace(go.Scatter(
                x=trend.index, y=trend,
                name=loc,
                line=dict(color=color, width=1, dash='dot'),
                opacity=0.25,
                legendgroup=f'{group}_trend',
                visible=show,
                showlegend=False
            ), row=1, col=1)
            trace_refs.append((period, len(fig.data) - 1))

        # --- Seasonality Snapshot (First full period only) ---
        season_part = seasonal_avg.dropna().iloc[:period]
        fig.add_trace(go.Scatter(
            x=list(range(1, len(season_part)+1)),
            y=season_part.values,
            name=f"{group} Seasonality",
            line=dict(color=color, dash='dot'),
            legendgroup=f'{group}_season',
            visible=show,
            showlegend=True
        ), row=2, col=1)
        trace_refs.append((period, len(fig.data) - 1))

# --- Slider Buttons ---
for period in rolling_options:
    vis = [False] * len(fig.data)
    for (p, idx) in trace_refs:
        if p == period:
            vis[idx] = True
    slider_steps.append(dict(
        method="update",
        label=f"{period}d",
        args=[
            {"visible": vis},
            {"title": f"STL Decomposition – 3x3 Extent – Period = {period} days"}
        ]
    ))

# --- Layout ---
fig.update_layout(
    sliders=[dict(
        active=0,
        currentvalue={"prefix": "STL Window: "},
        pad={"t": 50},
        y=1.12,
        x=0.6,
        xanchor="left",
        len=0.4,
        steps=slider_steps
    )],
    height=700,
    width=1200,
    paper_bgcolor='rgba(0,0,0,0)', # transparent background 
    title='STL Decomposition – 3x3 Extent – Period = 7 days',
    showlegend=True,
    legend=dict(
        orientation="v",
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top"
    )
)

fig.update_xaxes(title="Date", row=1, col=1)
fig.update_xaxes(title="Day within STL Cycle", row=2, col=1)
fig.update_yaxes(title="Trend", row=1, col=1)
fig.update_yaxes(title="Seasonality", row=2, col=1)

fig.show()

KeyError: 'group'

In [20]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

rows = []

for period in [1, 4, 7, 14, 30]:
    for group in ['Urban Core', 'Residential', 'Dark']:
        df_group = df[(df['group'] == group) & (df['extent'] == '3x3')]
        ts = df_group.groupby('date')['Gap_mean'].mean().dropna()

        if len(ts) < period * 2:
            continue

        try:
            decomposition = seasonal_decompose(ts, model='additive', period=period)
        except:
            continue

        trend = decomposition.trend.dropna()
        seasonal = decomposition.seasonal.dropna()

        # Format amplitude as "mean ± sd"
        seasonal_std = round(seasonal.std(), 2)
        amplitude_str = f"{round(seasonal_std*2, 2)} ± {seasonal_std}"

        rows.append({
            "Group": group,
            "Window (days)": period,
            "Trend Mean": round(trend.mean(), 2),
            "Trend SD": round(trend.std(), 2),
            "Seasonality Amplitude (±SD)": amplitude_str
        })

summary_df = pd.DataFrame(rows)
summary_df

,Group,Window (days),Trend Mean,Trend SD,Seasonality Amplitude (±SD)
0,Urban Core,1,17.89,3.28,0.0 ± 0.0
1,Residential,1,4.12,0.69,0.0 ± 0.0
2,Dark,1,0.15,0.09,0.0 ± 0.0
3,Urban Core,4,17.90,2.71,0.58 ± 0.29
4,Residential,4,4.12,0.52,0.1 ± 0.05
5,Dark,4,0.15,0.06,0.02 ± 0.01
6,Urban Core,7,17.89,2.63,0.48 ± 0.24
7,Residential,7,4.12,0.48,0.14 ± 0.07
8,Dark,7,0.15,0.06,0.02 ± 0.01
9,Urban Core,14,17.88,2.51,0.68 ± 0.34
